In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
import time
import numpy as np
import plotly.express as px
from utils import *

torch.manual_seed(10)


def visualise_tensor(data):
    data = data.detach().numpy()

    colormap = 'RdBu_r'
    fig = px.imshow(data, color_continuous_scale=colormap, zmin=-1, zmax=1)
    fig.show()
    fig.data = ()

weight_penalty_coefficient = 0.01

input_size = 20
hidden_layer_size= 5
output_size = input_size
learning_rate = 0.01
max_iters = 1000
dataset_size = 1024*32
sparsity = 0.1
batch_size = 32

decay_factor = 0.95
iters = []
loss_value = []
val_loss_value = []

imp_vector =torch.tensor([0.9**i for i in range(input_size)])
#imp_vector = torch.rand(input_size)


def sample_data(split = 'train'):
    if split != 'train':
        curr_data = validation_dataset
    else:
        curr_data = training_dataset
    sparsity_vector = sparsity
    a, b = curr_data.shape
    
    sparsity_vector = torch.tensor(sparsity_vector, device=curr_data.device)
    sparsity_vector = sparsity_vector.view(1, -1)
    sparsity_mat = torch.bernoulli(torch.full((a, b), 1.0) * sparsity_vector)
    
    X = curr_data * sparsity_mat
    return X, X


        

training_dataset = torch.rand(dataset_size, input_size)
validation_dataset = torch.rand(int(dataset_size/2), input_size)
output = input


model = SuperpositionModel(SuperPositionConfig(input_size, hidden_layer_size, imp_vector))

LOAD_MODEL = False
PATH = "models/basic_linear_0.99.bin"
if LOAD_MODEL:
    model = SuperpositionModel(SuperPositionConfig())
    model.load_state_dict(torch.load(PATH))
    model.eval()
else:
    optimizer = torch.optim.Adam(model.parameters() , learning_rate, weight_decay= 1e-5)
    start_time = time.time()
    step_value = max_iters/20
    for i in range(1, max_iters):
        X, Y = sample_data()
        logits, loss, _ = model(X, Y)
    
        if i%step_value ==0 :
            model.eval()
            with torch.no_grad():
                X_V,Y_V = sample_data(split = 'val')
                _, val_loss, _ = model(X_V, Y_V)
                iters.append(i)
                loss_value.append(loss.item())
                val_loss_value.append(val_loss.item())
                print(f"iter:{i} training loss: {loss.item()}, val loss: {val_loss.item()}")
            model.train()

        optimizer.zero_grad(set_to_none=True)
        #loss+= get_l1_penalty()
        loss.backward()
        optimizer.step()


    end_time = time.time()
    print(f"Took {end_time-start_time}s for {max_iters} epochs")

    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.plot(iters,loss_value, color='blue', label="Training")
    plt.plot(iters, val_loss_value, "red", label = "validation")
    plt.legend()
    plt.show()

/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Disabled automatic differentiation


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [17]:
encoder_w = model.encoder.weight.data
decoder_w = model.decoder.weight.data

# visualise_tensor(encoder_w.t())
# visualise_tensor(decoder_w.t())
from utils import *
imshow(decoder_w@encoder_w)
# visualise_tensor(model.decoder.bias.data.reshape(-1,1))

NameError: name 'imshow' is not defined

In [7]:
x,y = sample_data()

In [15]:
data = torch.tensor([])
original = torch.tensor([])
for i in range(10):
    x,y = sample_data()
    original = torch.cat((original, x))
    _,_,hidden = model(x,x)
    data = torch.cat((data,hidden))
    

torch.save(data, "data/hidden.pt")
torch.save(original, "data/original.pt")
PATH = "models/superpositon.pt"

torch.save(model.state_dict(), PATH)



In [39]:
LOAD_MODEL = False


In [8]:
model = SuperpositionModel(SuperPositionConfig(input_size, hidden_layer_size, imp_vector))

In [10]:
a, b, c = model(X,X)

In [12]:
b.shape

torch.Size([])

In [20]:
a.shape

torch.Size([65536, 20])

In [9]:
torch.rand(10)

tensor([0.6576, 0.5636, 0.9348, 0.4211, 0.1432, 0.0944, 0.8426, 0.0601, 0.8363,
        0.3298])